In [1]:
# set the working directory
import os as os
os.chdir("C:/Users/User/Desktop/18-2/DM/data")
print(os.getcwd())

### importing data function ###
def read():
    import pandas as pd
    name = input("Enter the data file name (with extension name) : ")
    fm = input("Select the data coding format(1='a b c' or 2='a,b,c'): " )
    if fm == '1':
        form = " "
    elif fm == '2':
        form = ","
    return pd.read_csv(name, sep=form, header=None)


### Regression function ###
def regression():
    
    # import packages
    import numpy as np
    import numpy.linalg as lin
    import pandas as pd
    
    # prompt user to enter the data information
    data = read()
    
    # import data
    response = data[num]
    explanatory = data.drop(num, axis=1)
    
    # design matrix
    n = data.shape[0]
    p = data.shape[1]-1
    one = pd.DataFrame(np.ones((n,1)))
    I = np.eye(n)
    X = pd.concat([pd.DataFrame(one),explanatory], axis=1)
    Y = response
    H = X.dot(lin.inv(X.T.dot(X))).dot(X.T)
    H0 = one.dot(lin.inv(one.T.dot(one))).dot(one.T)
    inv = lin.inv(X.T.dot(X))
    b = inv.dot(X.T).dot(Y).round(3)

    # multiple regression result
    yhat = H.dot(Y).round(1)
    SST = Y.T.dot(I-H0).dot(Y)
    SSE = Y.T.dot(I-H).dot(Y)
    Rsquare = round(1 - SSE/SST, 4)
    MSE = round(SSE/(n-p), 4)

    # output file name
    outputname = input("Write the output file name you want to save (without extension name) : ")
    outputname = outputname+'.txt'

    # outport the result
    with open(outputname,"w") as text_file:
        
        print("Coefficients", file=text_file)
        print("-------------", file=text_file)
        for i in range(p+1):
            if i==0:
                print("Constant:", b[i],sep="   ", file=text_file)
            else: 
                print("Beta",i,":   ",b[i],sep="", file=text_file)
        print("",file=text_file)

        print("ID, Actual values, Fitted values", file=text_file)
        print("--------------------------------", file=text_file)
        for i in range(n):
            print(i+1, Y[i], yhat[i], sep=", ", file=text_file)
        print("",file=text_file)
        
        print("Model Summary", file=text_file)
        print("-------------", file=text_file)
        print("R-square = ", Rsquare, sep="", file=text_file)
        print("MSE = ", MSE, sep="", file=text_file)

### Classification function ###
def classification() :
    # import packages
    import math
    import pandas as pd
    import numpy as np
    import numpy.linalg as lin
    from fractions import Fraction as frac

    # prompt user to enter the data
    print('Importing TRAINING dataset')
    train = read()
    print('Importing TEST dataset')
    test = read()
    num = int(input("Enter which column the response variable is recorded: "))-1

    # choose (i)LDA or (ii)QDA
    choice = int(input("Enter 1 to implement LDA or enter 2 to implement QDA"))
    
    nclass = len(train[num].unique()) # Assume that values of the class variable are integers starting with 1
    print(' It has',nclass,'Classes.','\n', 'Input each priors with ascending order of Class index.','\n',
          'ex) Priors of 3 class size = 1/3, 1/3, 1/3','\n')
    prior = input('Priors : ').split(',')
    prior = list(map(lambda x: frac(x), prior))
    if not len(prior) == nclass :
        prior = [1/nclass]*nclass
        print("Your prior input does not correscond with class size.")
        print("So equal prior is given such as", prior)
    elif not sum(prior)==1 :
        prior = [1/nclass]*nclass
        print("Sum of your prior input is not equal to 1.")
        print("So equal prior is given such as", prior)
        
    # Basic vectors  
    N = train.shape[0]
    n = test.shape[0]
    np = train[num].groupby(train[num]).count()
    x = train.drop(num, axis=1).T
    x_test = test.drop(num, axis=1).T
    y = train[num]
    y_test = test[num]
    y_pred = []
    y_test_pred = []
    prior = pd.DataFrame(prior)
    
    means = train.groupby(train[num]).mean().T
    cov = train.groupby(train[num]).cov()
    covs=[]
    sp = 0
    for i in range(1, nclass+1):
        covs.append(cov.loc[i])
        sp = sp + (np[i]-1)*cov.loc[i]/(N-nclass)
    
    # (i) LDA
    if choice == 1 :
        dresub=[]
        dtest=[]

        for i in range(1, nclass+1):
            t0 = (means[i].T).dot(lin.inv(sp)).dot(x)
            t1 = (means[i].T).dot(lin.inv(sp)).dot(x_test)
            t2 = (-0.5)*means[i].T.dot(lin.inv(sp)).dot(means[i])+math.log(prior.T[i-1])
            t3 = (-0.5)*means[i].T.dot(lin.inv(sp)).dot(means[i])+math.log(prior.T[i-1])
            dresub.append(t0+t2)
            dtest.append(t1+t3)
        y_pred = pd.DataFrame(dresub).idxmax()+1
        y_test_pred = pd.DataFrame(dtest).idxmax()+1

    # (ii) QDA
    elif choice == 2 :
        import numpy as np
        dresub = np.zeros((nclass,N))
        dtest = np.zeros((nclass,n))

        for i in range(0, nclass):
            for j in range(0, N):
                t0 = (-0.5)*math.log(lin.det(cov.loc[i+1]))
                t1 = (-0.5)*((x[j]-means[i+1]).T).dot(lin.inv(cov.loc[i+1])).dot(x[j]-means[i+1]) + math.log(prior.T[i])
                dresub[i,j]= t0+t1
            for j in range(0, n):
                t2 = (-0.5)*((x_test[j]-means[i+1]).T).dot(lin.inv(cov.loc[i+1])).dot(x_test[j]-means[i+1]) + math.log(prior.T[i])
                dtest[i,j]= t0+t2

        y_pred = pd.DataFrame(dresub).idxmax()+1
        y_test_pred = pd.DataFrame(dtest).idxmax()+1
    
    # Crosstable
    import numpy as np
    cross_res = pd.crosstab(y, y_pred, colnames=[''])
    cross_res.index = np.array(["Actual","Class",""]).repeat([1,1,nclass-2])
    accuracy_res = np.trace(cross_res)/N

    cross_t = pd.crosstab(y_test, y_test_pred, colnames=[''])
    cross_t.index = np.array(["Actual","Class",""]).repeat([1,1,nclass-2])
    accuracy_t = np.trace(cross_t)/n
    
    # output file name
    outputname = input("Write the output file name you want to save (without extension name) : ")
    outputname = outputname+'.txt'
    
    # outport the result
    with open(outputname,"w") as text_file:

        print('ID, Actual class, Resub pred', file=text_file)
        print('-----------------------------', file=text_file)
        for i in range(N):
             print(i+1, y[i], y_pred[i], sep=', ', file=text_file)
        print('',file=text_file)
        print('Confusion Matrix (Resubstitution)', file=text_file)
        print('----------------------------------', file=text_file)
        print(cross_res, file=text_file)
        print("",file=text_file)
        print("Model Summary (Resubstitution)", file=text_file)
        print('------------------------------', file=text_file)
        print("Overall accuracy = ", accuracy_res.round(3), sep='', file=text_file)
        print('', file=text_file)

        print('ID, Actual class, Test pred', file=text_file)
        print('-----------------------------', file=text_file)
        for i in range(n):
            print(i+1, y_test[i], y_test_pred[i], sep=', ', file=text_file)
        print('',file=text_file)
        print('Confusion Matrix (Test)', file=text_file)
        print('----------------------------------', file=text_file)
        print(cross_t, file=text_file)
        print("",file=text_file)
        print("Model Summary (Test)", file=text_file)
        print('------------------------------', file=text_file)
        print("Overall accuracy = ", accuracy_t.round(3), sep='', file=text_file)
        print('', file=text_file)
        
        
def HW3() : 
    method = input('Enter 1 to use Regression, Enter 2 to use Classification')
    if method == '1' :
        regression()
    else :
        classification()

C:\Users\User\Desktop\18-2\DM\data


In [4]:
HW3()

Enter 1 to use Regression, Enter 2 to use Classification2
Importing TRAINING dataset
Enter the data file name (with extension name) : veh.dat
Select the data coding format(1='a b c' or 2='a,b,c'): 2
Importing TEST dataset
Enter the data file name (with extension name) : vehtest.dat
Select the data coding format(1='a b c' or 2='a,b,c'): 2
Enter which column the response variable is recorded: 19
Enter 1 to implement LDA or enter 2 to implement QDA2
 It has 4 Classes. 
 Input each priors with ascending order of Class index. 
 ex) Priors of 3 class size = 1/3, 1/3, 1/3 

Priors : 1/4,1/4,1/4,1/4
Write the output file name you want to save (without extension name) : HW3KimDA_QDA_py_output
